<a href="https://colab.research.google.com/github/masaki-cucu/Kaggle/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TensorFlowとそのデータセットライブラリのインポート
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_datasets as tfds

TensorFlow 2.x selected.


In [0]:
# データセットの指定と分割
dataset = tfds.load('mnist', as_supervised=True)
mnist_train, mnist_test = dataset['train'], dataset['test']

In [0]:
# マップ関数（配列に対して中身を一気に変換するための関数）を定義
# ここではimageを実数に変換した後最大値1.0になるように除算している
# NNへの入力は0.0～1.0にスケーリングされているのが望ましいため
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

# 各データに上記マップ関数を適用。訓練データはランダムに混ぜている
# shuffleに引数を与えることで実行ごとに同じ混ざり方をするようにしている（再現性のため）
mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

In [0]:
# Kerasを使用する
# Dense：通常の全結合NN
# Flatten:入力を平坦化するためのもの
# Model:学習モデルの定義
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

# Modelを継承して今回使用する学習モデルを定義する
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # 入力層をFlattenで宣言することで入力を一列にする
        self.flatten = Flatten() # (28, 28, 1) -> 784
        # 全結合層 ユニット数128（数は雰囲気で決めていい）
        self.d1 = Dense(128, activation = 'relu') # 784 -> 128
        # 全結合層 ユニット数10（0～9を一個ずつ）
        self.d2 = Dense(10, activation = 'softmax') # 128 -> 10

    # initで宣言した各層をcall関数でつなげる
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# MyModel型のmodel変数を宣言
model = MyModel()

In [0]:
# 損失関数の定義
# SparseCategoricalCrossentropyはOne-hot型になってない教師データでもいい感じに解釈してくれるやつ
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# 学習のための勾配法はAdamという方法を選択
# どの勾配法が良いかは人の好みも大きいと思うけど最近はAdamが人気っぽい
# ほかのライブラリにもAdamはある
optimizer = tf.keras.optimizers.Adam()

In [0]:
# ログ出力用の損失Lossと正確性Accuracy
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [0]:
# 訓練とテスト用それぞれで学習または予測の1ステップの関数を定義
# @tf.functionを頭につけると処理が早くなる

@tf.function
def train_step(image, label):
    # tensorflowのGradientTape（勾配を求めるためのクラス）を実行出来たらそれをtapeとして以下の処理
    with tf.GradientTape() as tape:
        # modelにimageを入力することで結果を予想
        predictions = model(image)
        # 予想と教師との差（損失・誤差）を求める
        loss = loss_object(label, predictions)
    # 損失とモデル内の学習可能な変数リストから学習すべき勾配を求める
    gradients = tape.gradient(loss, model.trainable_variables)
    # 勾配と学習可能変数をoptimizerに代入することで学習
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(label, predictions)

# 予測して損失（誤差）を求めるだけ
@tf.function
def test_step(image, label):
    predictions = model(image)
    t_loss = loss_object(label, predictions)

    test_loss(t_loss)
    test_accuracy(label, predictions)

In [0]:
# 学習の実行
# 5エポック（5周）で大体80秒くらい？
EPOCHS = 5

for epoch in range(EPOCHS):
    for image, label in mnist_train:
        train_step(image, label)

    for test_image, test_label in mnist_test:
        test_step(test_image, test_label)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

Epoch 1, Loss: 0.25917848944664, Accuracy: 92.63333129882812, Test Loss: 0.14332175254821777, Test Accuracy: 95.67000579833984
Epoch 2, Loss: 0.18677496910095215, Accuracy: 94.63249969482422, Test Loss: 0.12067759782075882, Test Accuracy: 96.29000091552734
Epoch 3, Loss: 0.1499626785516739, Accuracy: 95.66388702392578, Test Loss: 0.10951710492372513, Test Accuracy: 96.6433334350586
Epoch 4, Loss: 0.12681038677692413, Accuracy: 96.30583190917969, Test Loss: 0.1043759286403656, Test Accuracy: 96.80999755859375
Epoch 5, Loss: 0.11004895716905594, Accuracy: 96.77967071533203, Test Loss: 0.09885060787200928, Test Accuracy: 96.98600006103516
